<a href="https://colab.research.google.com/github/tazar09/h3_working_repo/blob/main/heroes_3_army_comparator_08_feb2026.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
#!/usr/bin/env python3
!pip install fpdf2
import math
from collections import defaultdict
from typing import Dict, List, Optional, Tuple
from dataclasses import dataclass
from enum import Enum
from fpdf import FPDF # Import the PDF library

# ============================================================================
# DATA MODELS & LOGIC (Kept exactly as verified before)
# ============================================================================

HERO_STAT_MULTIPLIER = 0.05

class UnitType(Enum):
    INFANTRY = "Infantry"
    SHOOTER = "Shooter"
    FLYER = "Flyer"
    ARTILLERY = "Artillery"

@dataclass
class Creature:
    name: str
    town: str
    ai_value: int
    fight_value: int
    cost: int
    unit_type: UnitType
    speed: int

@dataclass
class HeroStats:
    name: str
    attack: int
    defense: int
    @property
    def multiplier(self) -> float:
        return math.sqrt((1 + HERO_STAT_MULTIPLIER * self.attack) * (1 + HERO_STAT_MULTIPLIER * self.defense))

@dataclass
class ArmyAnalysis:
    hero: HeroStats
    units: List[dict]
    total_ai_scaled: float
    total_fight_scaled: float
    total_gold: int
    type_breakdown_ai: Dict[str, float]
    town_breakdown_ai: Dict[str, float]
    avg_speed: float
    fastest_unit: Tuple[str, int]
    slowest_unit: Tuple[str, int]
    top_3_avg_speed: float

class CreatureDatabase:
    def __init__(self, raw_data: str):
        self.creatures: Dict[str, Creature] = {}
        for line in raw_data.strip().split('\n')[1:]:
            p = [x.strip() for x in line.split(',')]
            if len(p) < 7: continue
            self.creatures[p[0].lower()] = Creature(p[0], p[1], int(p[2]), int(p[3]), int(p[4]), UnitType(p[5]), int(p[6]))
    def get(self, name: str) -> Optional[Creature]:
        return self.creatures.get(name.lower())

class ArmyAnalyzer:
    def analyze(self, army_dict: Dict[str, int], hero: HeroStats, db: CreatureDatabase) -> ArmyAnalysis:
        units_list, t_ai_base, t_fight_base, t_gold = [], 0.0, 0.0, 0
        type_ai, town_ai, mult = defaultdict(float), defaultdict(float), hero.multiplier
        for name, count in army_dict.items():
            c = db.get(name)
            if not c: continue
            ai_val, f_val, g_val = c.ai_value * count, c.fight_value * count, c.cost * count
            units_list.append({'name': c.name, 'count': count, 'ai': ai_val, 'fight': f_val, 'gold': g_val, 'speed': c.speed, 'type': c.unit_type, 'town': c.town})
            t_ai_base += ai_val; t_fight_base += f_val; t_gold += g_val
            type_ai[c.unit_type.value] += ai_val * mult; town_ai[c.town] += ai_val * mult
        combat_units = [u for u in units_list if u['type'] != UnitType.ARTILLERY]
        speeds = [u['speed'] for u in combat_units]
        avg_s = sum(speeds) / len(speeds) if speeds else 0
        sorted_speeds = sorted(speeds, reverse=True)
        top_3 = sum(sorted_speeds[:3]) / min(len(sorted_speeds), 3) if speeds else 0
        fastest = max(combat_units, key=lambda x: x['speed']) if combat_units else {'name': 'N/A', 'speed': 0}
        slowest = min(combat_units, key=lambda x: x['speed']) if combat_units else {'name': 'N/A', 'speed': 0}
        return ArmyAnalysis(hero, units_list, t_ai_base * mult, t_fight_base * mult, t_gold, dict(type_ai), dict(town_ai), avg_s, (fastest['name'], fastest['speed']), (slowest['name'], slowest['speed']), top_3)

# ============================================================================
# UPDATED REPORT GENERATOR WITH PDF SUPPORT
# ============================================================================

class ReportGenerator:
    def __init__(self):
        self.output_text = []

    def log(self, text):
        print(text)
        self.output_text.append(text)

    def generate(self, a1: ArmyAnalysis, a2: ArmyAnalysis):
        n1, n2 = a1.hero.name, a2.hero.name
        def calc_diff(v1, v2): return ((v1 / v2) - 1) * 100 if v2 > 0 else 0

        # 1. Composition
        for a in [a1, a2]:
            self.log(f"\n--- {a.hero.name} Army Composition ---")
            header = f"{'Unit':<22} {'Count':<7} {'AI Value':<12} {'Fight Value':<12} {'Speed':<6}"
            self.log(header)
            self.log("-" * 65)
            for u in a.units:
                self.log(f"{u['name']:<22} {u['count']:<7,} {u['ai']:<12,} {u['fight']:<12,} {u['speed']:<6}")

        # 2. Main Comparison
        self.log(f"\n--- Army Comparison ---")
        self.log(f"{'Metric':<30} {n1:<18} {n2:<18} {'Diff %':<10}")
        self.log("-" * 85)
        metrics = [
            ("Total AI Strength", a1.total_ai_scaled, a2.total_ai_scaled),
            ("Total Fight Strength", a1.total_fight_scaled, a2.total_fight_scaled),
            ("Total Gold Cost", float(a1.total_gold), float(a2.total_gold)),
            ("Average Army Speed", a1.avg_speed, a2.avg_speed),
            ("Top 3 Units Avg Speed", a1.top_3_avg_speed, a2.top_3_avg_speed)
        ]
        for label, v1, v2 in metrics:
            diff = calc_diff(v1, v2)
            val1 = f"{int(v1):,}" if v1 > 1000 else f"{v1:.2f}"
            val2 = f"{int(v2):,}" if v2 > 1000 else f"{v2:.2f}"
            self.log(f"{label:<30} {val1:<18} {val2:<18} {diff:>+8.2f}%")

        # 3. Breakdowns
        self._print_bkdn("Breakdown by Unit Type (AI)", a1.type_breakdown_ai, a2.type_breakdown_ai, a1.total_ai_scaled, a2.total_ai_scaled, n1, n2)
        self._print_bkdn("Breakdown by Town (AI)", a1.town_breakdown_ai, a2.town_breakdown_ai, a1.total_ai_scaled, a2.total_ai_scaled, n1, n2)

        # 4. Move Order
        self.log(f"\n--- Battle Move Order Simulation ---")
        combined = sorted([(u['speed'], a1.hero.name, u['name']) for u in a1.units if u['type'] != UnitType.ARTILLERY] +
                          [(u['speed'], a2.hero.name, u['name']) for u in a2.units if u['type'] != UnitType.ARTILLERY], key=lambda x: x[0], reverse=True)
        self.log(f"{'Order':<7} {'Unit Name':<25} {'Owner':<15} {'Speed':<5}")
        for i, (spd, owner, name) in enumerate(combined, 1):
            self.log(f"{i:<7} {name:<25} {owner:<15} {spd:<5}")

        # 5. Comparison Notes
        self.log("\n" + "="*85 + "\nDETAILED COMPARISON NOTES\n" + "="*85)
        ai_leader = n1 if a1.total_ai_scaled > a2.total_ai_scaled else n2
        fi_leader = n1 if a1.total_fight_scaled > a2.total_fight_scaled else n2
        self.log(f"* Power Balance: {ai_leader} leads in AI Strength ({abs(calc_diff(a1.total_ai_scaled, a2.total_ai_scaled)):.2f}%), "
                 f" \n{fi_leader} leads in Fight Strength ({abs(calc_diff(a1.total_fight_scaled, a2.total_fight_scaled)):.2f}%).")

        m_lead = n1 if a1.avg_speed > a2.avg_speed else n2
        self.log(f"* Mobility Dominance: {m_lead} has higher avg speed ({max(a1.avg_speed, a2.avg_speed):.2f}). "
                 f"Top Speed: {max(a1.fastest_unit[1], a2.fastest_unit[1])}.")

        self._save_to_pdf()

    def _print_bkdn(self, title, d1, d2, tot1, tot2, n1, n2):
        self.log(f"\n--- {title} ---")
        self.log(f"{'Category':<15} {n1 + ' Val':<15} {n1 + ' %':<10} {n2 + ' Val':<15} {n2 + ' %':<10}")
        self.log("-" * 75)
        keys = sorted(set(list(d1.keys()) + list(d2.keys())))
        for k in keys:
            v1, v2 = d1.get(k, 0), d2.get(k, 0)
            p1, p2 = (v1/tot1*100) if tot1>0 else 0, (v2/tot2*100) if tot2>0 else 0
            self.log(f"{k:<15} {int(v1):<15,} {p1:<10.2f} {int(v2):<15,} {p2:<10.2f}")

    def _save_to_pdf(self):
        pdf = FPDF()
        pdf.add_page()
        pdf.set_font("Courier", size=10) # Fixed-width font for tables
        for line in self.output_text:
            pdf.cell(0, 5, txt=line, ln=True)
        pdf.output("HoMM3_Army_Report.pdf")
        print("\n>>> PDF Report generated: HoMM3_Army_Report.pdf")


# ============================================================================
# RUNTIME DATA & MAIN
# ============================================================================

RAW_CREATURE_DATA = """Creature,Town,AI,Fight,Gold_Cost,Type
Azure Dragon,Neutral,78845,56315,30000,Flyer,19
Crystal Dragon,Neutral,39338,30260,20000,Flyer,16
Faerie Dragon,Neutral,30501,16317,10000,Infantry,15
Rust Dragon,Neutral,26433,24030,15000,Flyer,17
Archangel,Castle,8776,6033,5000,Flyer,18
Black Dragon,Dungeon,8721,6783,4000,Flyer,15
Gold Dragon,Rampart,8613,6220,4000,Flyer,16
Titan,Tower,7500,5000,5000,Shooter,11
Haspid,Cove,7220,5554,4000,Infantry,12
Arch Devil,Inferno,7115,5243,4500,Flyer,17
Phoenix,Conflux,6721,4929,3000,Flyer,21
Juggernaut,Factory,6433,5361,3500,Infantry,7
Ancient Behemoth,Stronghold,6168,5397,3000,Infantry,9
Chaos Hydra,Fortress,5931,5272,3500,Infantry,7
Crimson Couatl,Factory,5341,3815,3500,Flyer,15
Devil,Inferno,5101,3759,2700,Flyer,11
Angel,Castle,5019,3585,3000,Flyer,12
Red Dragon,Dungeon,5003,3762,2500,Flyer,11
Ghost Dragon,Necropolis,4919,3228,3000,Flyer,14
Green Dragon,Rampart,4872,3654,2400,Flyer,10
Firebird,Conflux,4336,3097,2000,Flyer,15
Hydra,Fortress,4120,4120,2200,Infantry,5
Sea Serpent,Cove,3953,3162,2200,Infantry,9
Dreadnought,Factory,3879,3879,2200,Infantry,6
Giant,Tower,3718,3146,2000,Infantry,6
Couatl,Factory,3574,2521,2000,Flyer,11
Bone Dragon,Necropolis,3388,2420,1800,Flyer,11
Behemoth,Stronghold,3162,3162,1500,Infantry,6
Naga Queen,Tower,2840,2840,1600,Infantry,7
Dread Knight,Necropolis,2382,2029,1500,Infantry,9
Efreet Sultan,Inferno,2343,1802,1100,Flyer,13
Nix Warrior,Cove,2116,1763,1300,Infantry,7
Champion,Castle,2100,1800,1200,Infantry,9
Black Knight,Necropolis,2087,1753,1200,Infantry,7
War Unicorn,Rampart,2030,2030,950,Infantry,9
Magic Elemental,Conflux,2012,1724,1200,Infantry,9
Cavalier,Castle,1946,1668,1000,Infantry,7
Naga,Tower,1814,2016,1200,Infantry,5
Unicorn,Rampart,1806,1548,850,Infantry,7
Efreet,Inferno,1670,1413,900,Flyer,9
Scorpicore,Dungeon,1685,1248,1050,Flyer,11
Psychic Elemental,Conflux,1669,1431,950,Infantry,7
Manticore,Dungeon,1547,1215,850,Flyer,7
Cyclop King,Stronghold,1544,1110,1100,Shooter,8
Wyvern Monarch,Fortress,1518,1518,1100,Flyer,11
Bounty Hunter,Factory,1454,932,1100,Shooter,8
Nix,Cove,1415,1415,1000,Infantry,6
Gunslinger,Factory,1351,904,800,Shooter,7
Wyvern,Fortress,1350,1050,800,Flyer,7
Cyclop,Stronghold,1266,1055,750,Shooter,6
Pit Lord,Inferno,1224,1071,700,Infantry,7
OlgoiKhorkhoi,Factory,1220,894,650,Flyer,10
Enchanter,Neutral,1210,805,750,Shooter,9
Thunderbird,Stronghold,1106,869,700,Flyer,11
Power Lich,Necropolis,1079,889,600,Shooter,7
Minotaur King,Dungeon,1068,890,575,Infantry,8
Mighty Gorgon,Fortress,1028,1028,600,Infantry,6
Roc,Stronghold,1027,790,600,Flyer,7
Troll,Neutral,1024,1024,500,Infantry,7
Sandworm,Factory,991,793,575,Flyer,8
Sentinel Automaton,Factory,947,631,450,Infantry,9
Master Genie,Tower,942,748,600,Flyer,11
Fangarm,Neutral,929,929,650,Infantry,6
Gorgon,Fortress,890,890,525,Infantry,5
Genie,Tower,884,680,550,Flyer,7
Sorceress,Cove,852,655,565,Shooter,7
Lich,Necropolis,848,742,550,Shooter,6
Minotaur,Dungeon,835,835,500,Infantry,6
Cannon,Machine,825,900,3000,Artillery,0
Dendroid Soldier,Rampart,803,765,425,Infantry,4
Sea Witch,Cove,790,608,515,Shooter,6
Vampire Lord,Necropolis,783,652,500,Flyer,9
Diamond Golem,Neutral,775,775,750,Infantry,5
Pit Fiend,Inferno,765,765,500,Infantry,6
Zealot,Castle,750,500,450,Shooter,7
Greater Basilisk,Fortress,714,561,400,Infantry,7
Arch Mage,Tower,680,467,450,Shooter,7
Ogre Mage,Stronghold,672,672,400,Infantry,5
Automaton,Factory,669,398,350,Infantry,8
Ayssid,Cove,645,478,325,Flyer,11
Sea Dog,Cove,602,376,375,Shooter,7
Gold Golem,Neutral,600,600,500,Infantry,5
Ballista,Machine,600,650,1500,Artillery,0
Steel Golem,Neutral,597,597,400,Infantry,6
Crusader,Castle,588,588,400,Infantry,6
Sharpshooter,Neutral,585,415,400,Shooter,9
Monk,Castle,582,485,400,Shooter,5
Medusa Queen,Dungeon,577,423,330,Shooter,5
Mage,Tower,570,418,350,Shooter,5
Vampire,Necropolis,555,518,360,Flyer,6
Basilisk,Fortress,552,506,325,Infantry,5
Silver Pegasus,Rampart,532,418,275,Flyer,12
Satyr,Neutral,518,471,300,Infantry,7
Pegasus,Rampart,518,407,250,Flyer,8
Dendroid Guard,Rampart,517,690,350,Infantry,3
Medusa,Dungeon,517,379,300,Shooter,5
Stormbird,Cove,502,386,275,Flyer,9
Catapult,Machine,500,10,0,Artillery,0
Magma Elemental,Conflux,490,490,500,Infantry,6
Storm Elemental,Conflux,486,324,275,Shooter,8
Horned Demon,Inferno,480,480,270,Infantry,6
Energy Elemental,Conflux,470,360,400,Flyer,8
Royal Griffin,Castle,448,364,240,Flyer,9
Swordsman,Castle,445,445,300,Infantry,5
Demon,Inferno,480,480,250,Infantry,5
Ogre,Stronghold,416,520,300,Infantry,4
Iron Golem,Tower,412,412,200,Infantry,5
Corsair,Cove,407,311,275,Shooter,7
Ammo Cart,Machine,400,5,750,Artillery,0
Arrow Tower,Machine,400,5,0,Artillery,0
Cerberus,Inferno,392,308,250,Infantry,8
Ice Elemental,Conflux,380,315,375,Shooter,6
Evil Eye,Dungeon,367,245,280,Shooter,7
Hell Hound,Inferno,357,275,200,Infantry,7
Air Elemental,Conflux,356,324,250,Infantry,7
Griffin,Castle,351,324,200,Flyer,6
Fire Elemental,Conflux,345,345,350,Infantry,6
Nomad,Neutral,345,415,200,Infantry,7
Beholder,Dungeon,336,240,250,Shooter,5
Grand Elf,Rampart,331,195,225,Shooter,7
Earth Elemental,Conflux,330,415,400,Infantry,4
Wraith,Necropolis,315,252,230,Flyer,7
Water Elemental,Conflux,315,315,300,Infantry,5
Dragon Fly,Fortress,312,250,240,Flyer,13
Pirate,Cove,312,208,225,Shooter,6
First Aid Tent,Machine,300,10,100,Artillery,0
Engineer,Factory,278,232,170,Infantry,7
Mummy,Neutral,270,270,300,Infantry,5
Serpent Fly,Fortress,268,215,220,Flyer,9
Bellwether Armadillo,Factory,256,256,230,Infantry,6
Wight,Necropolis,252,231,200,Flyer,5
Stone Golem,Tower,250,339,150,Infantry,3
Magog,Inferno,240,210,175,Shooter,6
Orc Chieftain,Stronghold,240,200,165,Shooter,5
Harpy Hag,Dungeon,238,196,170,Flyer,9
Wood Elf,Rampart,234,195,200,Shooter,6
Battle Dwarf,Rampart,209,209,150,Infantry,5
Leprechaun,Neutral,190,190,100,Infantry,5
Wolf Raider,Stronghold,203,174,140,Infantry,8
Obsidian Gargoyle,Tower,201,155,160,Flyer,9
Armadillo,Factory,198,248,200,Infantry,4
Orc,Stronghold,192,175,150,Shooter,4
Mechanic,Factory,186,186,140,Infantry,6
Marksman,Castle,184,115,150,Shooter,6
Seaman,Cove,174,174,140,Infantry,6
Stone Gargoyle,Tower,165,150,130,Flyer6
Gog,Inferno,159,145,125,Shooter,4
Lizard Warrior,Fortress,209,174,140,Shooter,5
Crew Mate,Cove,155,155,110,Infantry,5
Harpy,Dungeon,154,140,130,Flyer,6
Boar,Neutral,145,145,150,Infantry,6
Dwarf,Rampart,138,194,120,Infantry,3
Centaur Captain,Rampart,138,115,90,Infantry,8
Rogue,Neutral,135,135,100,Infantry,6
Wolf Rider,Stronghold,130,130,100,Infantry,6
Zombie,Necropolis,128,160,125,Infantry,4
Archer,Castle,126,115,100,Shooter,4
Lizardman,Fortress,151,137,110,Shooter,4
Halberdier,Castle,115,115,75,Infantry,5
Centaur,Rampart,100,100,70,Infantry,6
Walking Dead,Necropolis,98,140,100,Infantry,3
Halfling Grenadier,Factory,95,76,60,Shooter,6
Sprite,Conflux,95,70,30,Flyer,9
Gnoll Marauder,Fortress,90,90,70,Infantry,5
Skeleton Warrior,Necropolis,85,85,70,Infantry,5
Infernal Troglodyte,Dungeon,84,84,65,Infantry,5
Pikeman,Castle,80,100,60,Infantry,4
Hobgoblin,Stronghold,78,65,50,Infantry,7
Halfling,Factory,75,60,40,Shooter,5
Oceanid,Cove,75,60,45,Flyer,8
Master Gremlin,Tower,66,55,40,Shooter,5
Familiar,Inferno,60,60,60,Infantry,7
Goblin,Stronghold,60,60,40,Infantry,5
Skeleton,Necropolis,60,75,60,Infantry,4
Troglodyte,Dungeon,59,73,50,Infantry,4
Nymph,Cove,57,52,35,Flyer,6
Gnoll,Fortress,56,70,50,Infantry,4
Pixie,Conflux,55,40,25,Flyer,7
Imp,Inferno,50,50,50,Infantry,5
Gremlin,Tower,44,55,30,Infantry,4
Peasant,Castle,15,15,10,Infantry,3
Kobold,Bulwark,54,74,40,Infantry,3
Kobold Foreman,Bulwark,84,80,60,Infantry,5
Mountain Ram,Bulwark,228,190,135,Infantry,7
Argali,Bulwark,250,208,170,Infantry,8
Snow Elf,Bulwark,370,264,230,Shooter,6
Steel Elf,Bulwark,526,292,260,Shooter,7
Yeti,Bulwark,504,504,325,Infantry,6
Yeti Runemaster,Bulwark,751,626,400,Infantry,7
Shaman,Bulwark,685,571,450,Shooter,5
Great Shaman,Bulwark,818,682,600,Shooter,6
Mammoth,Bulwark,1359,1359,850,Infantry,5
War Mammoth,Bulwark,1601,1601,1000,Infantry,6
Jotunn,Bulwark,4180,3483,2000,Infantry,8
Jotunn Warlord,Bulwark,6694,5355,3500,Infantry,10
"""

def get_input(idx, db):
    print(f"\n--- Hero {idx} ---")
    name = input("Name: ")
    att = int(input("Attack: "))
    dfn = int(input("Defense: "))
    army = {}
    while True:
        entry = input("Unit, Count (or 'done'): ")
        if entry.lower() == 'done': break
        try:
            n, c = [x.strip() for x in entry.split(',')]
            if db.get(n): army[n] = int(c)
        except: pass
    return HeroStats(name, att, dfn), army

def main():
    db = CreatureDatabase(RAW_CREATURE_DATA)
    h1_s, h1_a = get_input(1, db)
    h2_s, h2_a = get_input(2, db)
    ReportGenerator().generate(ArmyAnalyzer().analyze(h1_a, h1_s, db), ArmyAnalyzer().analyze(h2_a, h2_s, db))

if __name__ == "__main__":
    main()


--- Hero 1 ---
Name: RIssa
Attack: 28
Defense: 36
Unit, Count (or 'done'): Cyclop, 70
Unit, Count (or 'done'): Champion, 82
Unit, Count (or 'done'): Master Genie, 83
Unit, Count (or 'done'): Angel, 65
Unit, Count (or 'done'): War Mammoth, 49
Unit, Count (or 'done'): Naga Queen, 57
Unit, Count (or 'done'): Titan, 40
Unit, Count (or 'done'): done

--- Hero 2 ---
Name: Pyre
Attack: 35
Defense: 29
Unit, Count (or 'done'): Magog, 203
Unit, Count (or 'done'): War Mammoth, 51
Unit, Count (or 'done'): Efreet Sultan, 53
Unit, Count (or 'done'): Crimson Couatl, 19
Unit, Count (or 'done'): Arch Devil, 21
Unit, Count (or 'done'): Jotunn Warlord, 26
Unit, Count (or 'done'): Wyvern, 59
Unit, Count (or 'done'): done

--- RIssa Army Composition ---
Unit                   Count   AI Value     Fight Value  Speed 
-----------------------------------------------------------------
Cyclop                 70      88,620       73,850       6     
Champion               82      172,200      147,600      9    

/tmp/ipython-input-1061873203.py:167: DeprecationWarning: The parameter "txt" has been renamed to "text" in 2.7.6
  pdf.cell(0, 5, txt=line, ln=True)
/tmp/ipython-input-1061873203.py:167: DeprecationWarning: The parameter "ln" is deprecated since v2.5.2. Instead of ln=True use new_x=XPos.LMARGIN, new_y=YPos.NEXT.
  pdf.cell(0, 5, txt=line, ln=True)


In [ ]:
C6